# project X: fraudulent insurance claims (alternative)

## importing relevant libraries 

In [114]:
#starting pyspark 
import findspark
findspark.init()
findspark.find()


'C:\\spark-3.3.1-bin-hadoop2'

In [115]:
#importing relevant libraries 
import os
import findspark
import py4j
from pyspark.sql import SparkSession

from pyspark.sql.types import *
from pyspark.sql.functions import mean,countDistinct, avg,stddev, col, count, when, floor, ceil
from pyspark.sql.functions import split, col, regexp_extract, lit
import pyspark.sql.functions as F
from pyspark.sql.functions import year
from pyspark.sql.functions import month
from pyspark.sql.functions import dayofweek

from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns#for plotting

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.types import StructType

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import imblearn
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import validation_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier


In [116]:
#starting pyspark
spark = SparkSession.builder.appName("project_X").getOrCreate()
spark

## importing data 

In [117]:
claim = pd.read_csv('Data/insurance_fraud_data.csv')
claim_bi = pd.read_csv('Data/insurance_fraud_data.csv')

In [118]:
claim_ml_df = pd.DataFrame(claim)
claim_bi_df = pd.DataFrame(claim_bi)

## EDA

## No. of columns 

In [119]:
print('ML dataset columns: \n')
display(claim_ml_df.columns)
print('powerBI dataset columns: \n')
display(claim_bi_df.columns) 

ML dataset columns: 



Index(['claim_number', 'age_of_driver', 'gender', 'marital_status',
       'safty_rating', 'annual_income', 'high_education_ind',
       'address_change_ind', 'living_status', 'zip_code', 'claim_date',
       'claim_day_of_week', 'accident_site', 'past_num_of_claims',
       'witness_present_ind', 'liab_prct', 'channel',
       'policy_report_filed_ind', 'claim_est_payout', 'age_of_vehicle',
       'vehicle_category', 'vehicle_price', 'vehicle_color', 'vehicle_weight',
       'fraud'],
      dtype='object')

powerBI dataset columns: 



Index(['claim_number', 'age_of_driver', 'gender', 'marital_status',
       'safty_rating', 'annual_income', 'high_education_ind',
       'address_change_ind', 'living_status', 'zip_code', 'claim_date',
       'claim_day_of_week', 'accident_site', 'past_num_of_claims',
       'witness_present_ind', 'liab_prct', 'channel',
       'policy_report_filed_ind', 'claim_est_payout', 'age_of_vehicle',
       'vehicle_category', 'vehicle_price', 'vehicle_color', 'vehicle_weight',
       'fraud'],
      dtype='object')

## function to get data information

In [120]:
def explore(data):
    print('data head: \n')
    display(data.head())
    print('info: \n')
    display(data.info())
    print('description statistics: \n')
    display(data.describe())
    print('how many rows?: \n')
    display(len(data))
    print('How many nulls: \n')
    display(data.isna().sum())
    
    

## input data into function

In [121]:
explore(claim_ml_df)

data head: 



,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,liab_prct,channel,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud
0,1,46,M,1.0,85,38301,1,1,Rent,80006,...,74,Broker,0,7530.940993,9.0,Compact,12885.45235,white,16161.33381,0
1,3,21,F,0.0,75,30445,0,1,Rent,15021,...,79,Online,0,2966.024895,4.0,Large,29429.45218,white,28691.96422,0
2,4,49,F,0.0,87,38923,0,1,Own,20158,...,0,Broker,0,6283.888333,3.0,Compact,21701.18195,white,22090.94758,1
3,5,58,F,1.0,58,40605,1,0,Own,15024,...,99,Broker,1,6169.747994,4.0,Medium,13198.27344,other,38329.58106,1
4,6,38,M,1.0,95,36380,1,0,Rent,50034,...,7,Broker,0,4541.387150,7.0,Medium,38060.21122,gray,25876.56319,0


info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17998 entries, 0 to 17997
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   claim_number             17998 non-null  int64  
 1   age_of_driver            17998 non-null  int64  
 2   gender                   17998 non-null  object 
 3   marital_status           17993 non-null  float64
 4   safty_rating             17998 non-null  int64  
 5   annual_income            17998 non-null  int64  
 6   high_education_ind       17998 non-null  int64  
 7   address_change_ind       17998 non-null  int64  
 8   living_status            17998 non-null  object 
 9   zip_code                 17998 non-null  int64  
 10  claim_date               17998 non-null  object 
 11  claim_day_of_week        17998 non-null  object 
 12  accident_site            17998 non-null  object 
 13  past_num_of_claims       17998 non-null  int64  
 14  witness_presen

None

description statistics: 



,claim_number,age_of_driver,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,zip_code,past_num_of_claims,witness_present_ind,liab_prct,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_price,vehicle_weight,fraud
count,17998.000000,17998.000000,17993.000000,17998.000000,17998.000000,17998.000000,17998.000000,17998.000000,17998.000000,17866.000000,17998.000000,17998.000000,17981.000000,17990.000000,17998.000000,17998.000000,17998.000000
mean,14970.601622,43.695466,0.712722,73.562951,37367.655684,0.699189,0.577286,49875.595955,0.505001,0.232677,49.423269,0.600678,4975.792083,5.008060,23089.123114,23031.322385,0.156462
std,8659.940765,11.959819,0.452505,15.346807,2957.297249,0.458623,0.494004,29214.655149,0.955504,0.422550,33.678470,0.489773,2215.706510,2.258391,11988.429767,12052.385584,0.363303
min,1.000000,18.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,282.639432,0.000000,2457.329316,2429.429302,0.000000
25%,7479.250000,35.000000,0.000000,65.000000,35554.000000,0.000000,0.000000,20111.000000,0.000000,0.000000,17.000000,0.000000,3337.029436,3.000000,14279.574850,14164.122133,0.000000
50%,14965.500000,43.000000,1.000000,76.000000,37610.000000,1.000000,1.000000,50028.000000,0.000000,0.000000,50.000000,1.000000,4668.796318,5.000000,20948.879250,20838.150260,0.000000
75%,22467.750000,51.000000,1.000000,85.000000,39318.000000,1.000000,1.000000,80038.000000,1.000000,0.000000,81.000000,1.000000,6255.901066,6.000000,29562.232780,29430.446293,0.000000
max,30000.000000,229.000000,1.000000,100.000000,54333.000000,1.000000,1.000000,85083.000000,6.000000,1.000000,100.000000,1.000000,17218.345010,16.000000,127063.506000,123016.650400,1.000000


how many rows?: 



17998

How many nulls: 



claim_number                 0
age_of_driver                0
gender                       0
marital_status               5
safty_rating                 0
annual_income                0
high_education_ind           0
address_change_ind           0
living_status                0
zip_code                     0
claim_date                   0
claim_day_of_week            0
accident_site                0
past_num_of_claims           0
witness_present_ind        132
liab_prct                    0
channel                      0
policy_report_filed_ind      0
claim_est_payout            17
age_of_vehicle               8
vehicle_category             0
vehicle_price                0
vehicle_color                0
vehicle_weight               0
fraud                        0
dtype: int64

In [122]:
explore(claim_bi_df)

data head: 



,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,liab_prct,channel,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud
0,1,46,M,1.0,85,38301,1,1,Rent,80006,...,74,Broker,0,7530.940993,9.0,Compact,12885.45235,white,16161.33381,0
1,3,21,F,0.0,75,30445,0,1,Rent,15021,...,79,Online,0,2966.024895,4.0,Large,29429.45218,white,28691.96422,0
2,4,49,F,0.0,87,38923,0,1,Own,20158,...,0,Broker,0,6283.888333,3.0,Compact,21701.18195,white,22090.94758,1
3,5,58,F,1.0,58,40605,1,0,Own,15024,...,99,Broker,1,6169.747994,4.0,Medium,13198.27344,other,38329.58106,1
4,6,38,M,1.0,95,36380,1,0,Rent,50034,...,7,Broker,0,4541.387150,7.0,Medium,38060.21122,gray,25876.56319,0


info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17998 entries, 0 to 17997
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   claim_number             17998 non-null  int64  
 1   age_of_driver            17998 non-null  int64  
 2   gender                   17998 non-null  object 
 3   marital_status           17993 non-null  float64
 4   safty_rating             17998 non-null  int64  
 5   annual_income            17998 non-null  int64  
 6   high_education_ind       17998 non-null  int64  
 7   address_change_ind       17998 non-null  int64  
 8   living_status            17998 non-null  object 
 9   zip_code                 17998 non-null  int64  
 10  claim_date               17998 non-null  object 
 11  claim_day_of_week        17998 non-null  object 
 12  accident_site            17998 non-null  object 
 13  past_num_of_claims       17998 non-null  int64  
 14  witness_presen

None

description statistics: 



,claim_number,age_of_driver,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,zip_code,past_num_of_claims,witness_present_ind,liab_prct,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_price,vehicle_weight,fraud
count,17998.000000,17998.000000,17993.000000,17998.000000,17998.000000,17998.000000,17998.000000,17998.000000,17998.000000,17866.000000,17998.000000,17998.000000,17981.000000,17990.000000,17998.000000,17998.000000,17998.000000
mean,14970.601622,43.695466,0.712722,73.562951,37367.655684,0.699189,0.577286,49875.595955,0.505001,0.232677,49.423269,0.600678,4975.792083,5.008060,23089.123114,23031.322385,0.156462
std,8659.940765,11.959819,0.452505,15.346807,2957.297249,0.458623,0.494004,29214.655149,0.955504,0.422550,33.678470,0.489773,2215.706510,2.258391,11988.429767,12052.385584,0.363303
min,1.000000,18.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,282.639432,0.000000,2457.329316,2429.429302,0.000000
25%,7479.250000,35.000000,0.000000,65.000000,35554.000000,0.000000,0.000000,20111.000000,0.000000,0.000000,17.000000,0.000000,3337.029436,3.000000,14279.574850,14164.122133,0.000000
50%,14965.500000,43.000000,1.000000,76.000000,37610.000000,1.000000,1.000000,50028.000000,0.000000,0.000000,50.000000,1.000000,4668.796318,5.000000,20948.879250,20838.150260,0.000000
75%,22467.750000,51.000000,1.000000,85.000000,39318.000000,1.000000,1.000000,80038.000000,1.000000,0.000000,81.000000,1.000000,6255.901066,6.000000,29562.232780,29430.446293,0.000000
max,30000.000000,229.000000,1.000000,100.000000,54333.000000,1.000000,1.000000,85083.000000,6.000000,1.000000,100.000000,1.000000,17218.345010,16.000000,127063.506000,123016.650400,1.000000


how many rows?: 



17998

How many nulls: 



claim_number                 0
age_of_driver                0
gender                       0
marital_status               5
safty_rating                 0
annual_income                0
high_education_ind           0
address_change_ind           0
living_status                0
zip_code                     0
claim_date                   0
claim_day_of_week            0
accident_site                0
past_num_of_claims           0
witness_present_ind        132
liab_prct                    0
channel                      0
policy_report_filed_ind      0
claim_est_payout            17
age_of_vehicle               8
vehicle_category             0
vehicle_price                0
vehicle_color                0
vehicle_weight               0
fraud                        0
dtype: int64

## value counts, max and min 

In [123]:
print('safety rating: \n')
display('counts: ',claim_bi_df['safty_rating'].value_counts())
display('max: ', claim_bi_df['safty_rating'].max())
display('min: ', claim_bi_df['safty_rating'].min())

print('high education: \n')
display('counts: ', claim_bi_df['high_education_ind'].value_counts())
display('max: ', claim_bi_df['high_education_ind'].max())
display('min: ', claim_bi_df['high_education_ind'].min())

print('living_status: \n')
display('counts: ', claim_bi_df['living_status'].value_counts())
# display(claim_bi_df['safty_rating'].max())
# display(claim_bi_df['safty_rating'].min())

print('accident site: \n')
display('counts: ', claim_bi_df['accident_site'].value_counts())
# display(claim_bi_df['safty_rating'].max())
# display(claim_bi_df['safty_rating'].min())

print('past number of claims: \n')
display('counts: ', claim_bi_df['past_num_of_claims'].value_counts())
display('max: ', claim_bi_df['past_num_of_claims'].max())
display('min: ', claim_bi_df['past_num_of_claims'].min())

print('witness present: \n')
display('counts: ', claim_bi_df['witness_present_ind'].value_counts())
display('max: ', claim_bi_df['witness_present_ind'].max())
display('min: ', claim_bi_df['witness_present_ind'].min())

print('liability %: \n')
display('counts: ', claim_bi_df['liab_prct'].value_counts())
display('max: ', claim_bi_df['liab_prct'].max())
display('min: ', claim_bi_df['liab_prct'].min())

print('channel: \n')
display('counts: ', claim_bi_df['channel'].value_counts())
# display(claim_bi_df['safty_rating'].max())
# display(claim_bi_df['safty_rating'].min())

print('policy report filled: \n')
display('counts: ', claim_bi_df['policy_report_filed_ind'].value_counts())
display('max: ', claim_bi_df['policy_report_filed_ind'].max())
display('min: ', claim_bi_df['policy_report_filed_ind'].min())

print('vehicle category: \n')
display('counts: ', claim_bi_df['vehicle_category'].value_counts())
# display(claim_bi_df['vehicle_category'].max())
# display(claim_bi_df['vehicle_category'].min())

print('vehicle price: \n')
display('counts: ', claim_bi_df['vehicle_price'].value_counts())
display(claim_bi_df['vehicle_price'].max())
display(claim_bi_df['vehicle_price'].min())

print('vehicle color: \n')
display('counts: ', claim_bi_df['vehicle_price'].value_counts())
# display(claim_bi_df['vehicle_price'].max())
# display(claim_bi_df['vehcile_price'].min())

print('vehicle weight: \n')
display('counts: ', claim_bi_df['vehicle_weight'].value_counts())
display('max: ', claim_bi_df['vehicle_weight'].max())
display('min: ', claim_bi_df['vehicle_weight'].min())

print('fraud: \n')
display('counts: ', claim_bi_df['fraud'].value_counts())
display('max: ', claim_bi_df['fraud'].max())
display('min: ', claim_bi_df['fraud'].min())

print('estimated payout: \n')
display('counts: ', claim_bi_df['claim_est_payout'].value_counts())
display('max: ', claim_bi_df['claim_est_payout'].max())
display('min: ', claim_bi_df['claim_est_payout'].min())

print('vehicle age: \n')
display('counts: ', claim_bi_df['age_of_vehicle'].value_counts())
display('max: ', claim_bi_df['age_of_vehicle'].max())
display('min: ', claim_bi_df['age_of_vehicle'].min())

safety rating: 



'counts: '

84    565
80    557
85    555
81    548
83    532
     ... 
12      2
1       2
6       2
7       1
3       1
Name: safty_rating, Length: 100, dtype: int64

'max: '

100

...

4.0     3111
5.0     3101
6.0     2644
3.0     2561
7.0     1864
2.0     1524
8.0     1178
9.0      668
1.0      617
10.0     333
11.0     161
0.0      121
12.0      63
13.0      28
14.0      12
15.0       3
16.0       1
Name: age_of_vehicle, dtype: int64

'max: '

16.0

'min: '

0.0

## changing data types + rounding

In [124]:
#date 
claim_ml_df['claim_date'] = pd.to_datetime(claim_ml_df['claim_date'])
claim_bi_df['claim_date'] = pd.to_datetime(claim_bi_df['claim_date'])

In [125]:
#annual income salary rounded 
claim_bi_df['annual_income'] = round(claim_bi_df['annual_income'], 0)
claim_ml_df['annual_income'] = round(claim_ml_df['annual_income'], 0)

In [126]:
#vehicle price rounded 
claim_bi_df['vehicle_price'] = round(claim_bi_df['vehicle_price'], 0)
claim_ml_df['vehicle_price'] = round(claim_ml_df['vehicle_price'], 0)

In [127]:
#vehicle mass rounded 
claim_bi_df['vehicle_weight'] = round(claim_bi_df['vehicle_weight'], 2)
claim_ml_df['vehicle_weight'] = round(claim_ml_df['vehicle_weight'], 2)


In [128]:
claim_bi_df['claim_est_payout'] = round(claim_bi_df['claim_est_payout'], 0)
claim_ml_df['claim_est_payout'] = round(claim_ml_df['claim_est_payout'], 0)

## Data Cleaning and preprocessing: filling nulls + replacing values

### maritial status

In [129]:
display(claim_ml_df['marital_status'].value_counts()) #value count
display(claim_bi_df['marital_status'].value_counts())

1.0    12824
0.0     5169
Name: marital_status, dtype: int64

1.0    12824
0.0     5169
Name: marital_status, dtype: int64

In [130]:
claim_ml_df['marital_status'] = claim_ml_df['marital_status'].fillna(claim_ml_df['marital_status'].mode()[0]) #fill nulls
claim_bi_df['marital_status'] = claim_bi_df['marital_status'].fillna(claim_bi_df['marital_status'].mode()[0])

### witness present 

In [131]:
display(claim_ml_df['witness_present_ind'].value_counts()) #value count
display(claim_bi_df['witness_present_ind'].value_counts())

0.0    13709
1.0     4157
Name: witness_present_ind, dtype: int64

0.0    13709
1.0     4157
Name: witness_present_ind, dtype: int64

In [132]:
claim_ml_df['witness_present_ind'] = claim_ml_df['witness_present_ind'].fillna(claim_ml_df['witness_present_ind'].mode()[0]) # passed as argument to fill nulls 
claim_bi_df['witness_present_ind'] = claim_bi_df['witness_present_ind'].fillna(claim_bi_df['witness_present_ind'].mode()[0])

### claim: estimated payout

In [133]:
display(claim_ml_df['claim_est_payout'].value_counts()) #value counts
print('\n')
display(claim_bi_df['claim_est_payout'].value_counts())
print('\n')
print('max and min: \n')
display(claim_bi_df['claim_est_payout'].max()) #max and min
display(claim_ml_df['claim_est_payout'].min())

4475.0    13
3922.0    12
4249.0    11
4967.0    11
4210.0    11
          ..
8630.0     1
1555.0     1
2435.0     1
3637.0     1
9561.0     1
Name: claim_est_payout, Length: 7347, dtype: int64

4475.0    13
3922.0    12
4249.0    11
4967.0    11
4210.0    11
          ..
8630.0     1
1555.0     1
2435.0     1
3637.0     1
9561.0     1
Name: claim_est_payout, Length: 7347, dtype: int64



max and min: 



17218.0

283.0

In [134]:
claim_ml_df['claim_est_payout'] = claim_ml_df['claim_est_payout'].fillna(value = claim_ml_df['claim_est_payout'].mean()) #filling nulls with mean
claim_bi_df['claim_est_payout'] = claim_bi_df['claim_est_payout'].fillna(value = claim_bi_df['claim_est_payout'].mean())

### age of vehicle

In [135]:
#max and min 
print('bi max and min: \n')
display(claim_bi_df['age_of_vehicle'].max()) 
display(claim_bi_df['age_of_vehicle'].min()) 

print('ml max and min: \n')
display(claim_bi_df['age_of_vehicle'].max())
display(claim_ml_df['age_of_vehicle'].min())

bi max and min: 



16.0

0.0

ml max and min: 



16.0

0.0

In [136]:
#get mean
claim_ml_df['age_of_vehicle'] = claim_ml_df['age_of_vehicle'].fillna(value = claim_ml_df['age_of_vehicle'].mean()) #fill nulls 
claim_bi_df['age_of_vehicle'] = claim_bi_df['age_of_vehicle'].fillna(value = claim_bi_df['age_of_vehicle'].mean())

## salary

In [137]:
#max and min 
print('bi max and min: \n')
display(claim_bi_df['annual_income'].max()) 
display(claim_bi_df['annual_income'].min()) 
print('ml max and min: \n')
display(claim_ml_df['annual_income'].max())
display(claim_ml_df['annual_income'].min())

bi max and min: 



54333

-1

ml max and min: 



54333

-1

In [138]:
#assessing how many salaries are below $15000
claim_bi_df[claim_bi_df['annual_income'] < 15000]

,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,liab_prct,channel,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud
937,1542,49,F,1.0,39,-1,0,1,Rent,20124,...,98,Online,1,4311.0,1.0,Compact,17959.0,other,20932.77,0
3442,5734,50,M,0.0,55,-1,1,0,Own,20102,...,23,Phone,0,2888.0,8.0,Compact,33992.0,silver,32819.48,0
5449,9020,44,F,1.0,33,-1,0,1,Rent,50035,...,50,Broker,1,6652.0,6.0,Medium,32016.0,blue,21547.09,1
7347,12201,58,F,1.0,72,-1,0,1,Rent,20101,...,72,Phone,1,5421.0,3.0,Compact,16014.0,gray,4350.80,0
7467,12419,38,F,0.0,70,-1,0,0,Own,50009,...,90,Broker,1,6583.0,4.0,Large,29265.0,red,38451.44,0
8193,13601,83,F,0.0,87,-1,1,1,Rent,15025,...,15,Broker,0,6859.0,5.0,Compact,42317.0,other,25246.64,0
9385,15608,51,M,1.0,71,-1,0,1,Own,80014,...,33,Online,1,3892.0,4.0,Compact,17089.0,silver,23559.73,0
9628,16010,60,F,1.0,60,-1,1,1,Own,50038,...,78,Phone,1,2807.0,6.0,Large,32356.0,blue,17473.62,0
10539,17523,53,F,0.0,66,-1,1,1,Own,20104,...,81,Phone,1,1203.0,5.0,Compact,12693.0,silver,22501.40,0
11405,18945,45,F,1.0,83,-1,1,1,Rent,85013,...,3,Broker,0,6067.0,6.0,Medium,14766.0,white,24084.69,0


In [139]:
#replacing anything below $15000 with $15080.4 as the national USA living salary 
salary_clean_bi = []
salary_clean_ml = []

usa_min_sal = 15080.4

for sal in claim_ml_df['annual_income']:
    if sal < 15000.0:
        salary_clean_ml.append(usa_min_sal)
    else:
        salary_clean_ml.append(sal)
        
for sal1 in claim_bi_df['annual_income']:
    if sal1 < 15000.0:
        salary_clean_bi.append(usa_min_sal)
    else:
        salary_clean_bi.append(sal1)


In [140]:
display(salary_clean_bi)
display(salary_clean_ml)

[38301,
 30445,
 38923,
 40605,
 36380,
 40240,
 32952,
 36891,
 38069,
 34324,
 39699,
 40753,
 32194,
 36635,
 39322,
 38516,
 32592,
 38293,
 37609,
 36639,
 36876,
 42499,
 31796,
 34674,
 39119,
 52988,
 37855,
 36117,
 39512,
 40259,
 34654,
 36893,
 39496,
 39129,
 40599,
 41269,
 38508,
 30902,
 38483,
 32591,
 36387,
 41428,
 35551,
 37119,
 34025,
 39117,
 35561,
 37368,
 42336,
 33327,
 33993,
 42352,
 37140,
 31767,
 42785,
 38712,
 34326,
 34347,
 38701,
 36644,
 43577,
 36374,
 40616,
 40777,
 34958,
 34341,
 34327,
 42609,
 30897,
 41890,
 48905,
 37141,
 34675,
 40081,
 34958,
 40069,
 37855,
 41906,
 40934,
 37140,
 35546,
 38302,
 34676,
 31788,
 39701,
 39150,
 33660,
 38071,
 34949,
 32969,
 32578,
 32582,
 35278,
 40426,
 32590,
 33305,
 39328,
 38278,
 36382,
 37384,
 36891,
 30911,
 39692,
 40258,
 39526,
 39514,
 40600,
 41114,
 32191,
 36890,
 38066,
 40419,
 30896,
 31786,
 35828,
 36106,
 38717,
 34962,
 39895,
 40082,
 38696,
 36639,
 35268,
 40273,
 37138,


[38301,
 30445,
 38923,
 40605,
 36380,
 40240,
 32952,
 36891,
 38069,
 34324,
 39699,
 40753,
 32194,
 36635,
 39322,
 38516,
 32592,
 38293,
 37609,
 36639,
 36876,
 42499,
 31796,
 34674,
 39119,
 52988,
 37855,
 36117,
 39512,
 40259,
 34654,
 36893,
 39496,
 39129,
 40599,
 41269,
 38508,
 30902,
 38483,
 32591,
 36387,
 41428,
 35551,
 37119,
 34025,
 39117,
 35561,
 37368,
 42336,
 33327,
 33993,
 42352,
 37140,
 31767,
 42785,
 38712,
 34326,
 34347,
 38701,
 36644,
 43577,
 36374,
 40616,
 40777,
 34958,
 34341,
 34327,
 42609,
 30897,
 41890,
 48905,
 37141,
 34675,
 40081,
 34958,
 40069,
 37855,
 41906,
 40934,
 37140,
 35546,
 38302,
 34676,
 31788,
 39701,
 39150,
 33660,
 38071,
 34949,
 32969,
 32578,
 32582,
 35278,
 40426,
 32590,
 33305,
 39328,
 38278,
 36382,
 37384,
 36891,
 30911,
 39692,
 40258,
 39526,
 39514,
 40600,
 41114,
 32191,
 36890,
 38066,
 40419,
 30896,
 31786,
 35828,
 36106,
 38717,
 34962,
 39895,
 40082,
 38696,
 36639,
 35268,
 40273,
 37138,


## age of driver

In [141]:
#max and min 
print('bi max and min: \n')
display(claim_bi_df['age_of_driver'].max()) 
display(claim_bi_df['age_of_driver'].min()) 
print('ml max and min: \n')
display(claim_ml_df['age_of_driver'].max())
display(claim_ml_df['age_of_driver'].min())

bi max and min: 



229

18

ml max and min: 



229

18

In [142]:
#checking how many people are over 100
claim_bi_df[claim_bi_df['age_of_driver'] > 100]

,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,liab_prct,channel,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud
25,38,200,M,1.0,60,52988,1,1,Rent,85020,...,85,Phone,1,5782.0,5.0,Compact,21503.0,other,21437.34,0
70,111,133,M,1.0,60,48905,1,1,Own,15014,...,22,Broker,1,5923.0,7.0,Medium,7627.0,blue,24659.78,0
1217,2005,229,M,1.0,52,54333,1,0,Rent,15033,...,31,Broker,1,5025.0,6.0,Compact,29133.0,gray,19643.51,1
14078,23404,113,F,1.0,79,47273,1,0,Own,85033,...,93,Broker,0,5175.0,6.0,Large,19407.0,white,12325.66,0


In [143]:
# changing age over 100 to usa retirement age 

driver_age_clean_ml = []
driver_age_clean_bi = []
usa_retirement = 67

for age1 in claim_bi_df['age_of_driver']:
    if age1 > 100:
        driver_age_clean_ml.append(usa_retirement)
    else:
        driver_age_clean_ml.append(age1)

for age2 in claim_ml_df['age_of_driver']:
    if age2 > 100:
        driver_age_clean_bi.append(usa_retirement)
    else:
        driver_age_clean_bi.append(age2)

display(driver_age_clean_ml)
display(driver_age_clean_bi)

[46,
 21,
 49,
 58,
 38,
 56,
 27,
 40,
 45,
 31,
 53,
 59,
 25,
 39,
 51,
 47,
 26,
 46,
 43,
 39,
 40,
 70,
 24,
 32,
 50,
 67,
 44,
 37,
 52,
 56,
 32,
 40,
 52,
 50,
 58,
 62,
 47,
 22,
 47,
 26,
 38,
 63,
 35,
 41,
 30,
 50,
 35,
 42,
 69,
 28,
 30,
 69,
 41,
 24,
 72,
 48,
 31,
 31,
 48,
 39,
 78,
 38,
 58,
 59,
 33,
 31,
 31,
 71,
 22,
 66,
 67,
 41,
 32,
 55,
 33,
 55,
 44,
 66,
 60,
 41,
 35,
 46,
 32,
 24,
 53,
 50,
 29,
 45,
 33,
 27,
 26,
 26,
 34,
 57,
 26,
 28,
 51,
 46,
 38,
 42,
 40,
 22,
 53,
 56,
 52,
 52,
 58,
 61,
 25,
 40,
 45,
 57,
 22,
 24,
 36,
 37,
 48,
 33,
 54,
 55,
 48,
 39,
 34,
 56,
 41,
 29,
 46,
 34,
 55,
 45,
 53,
 32,
 54,
 40,
 41,
 52,
 56,
 34,
 35,
 36,
 38,
 52,
 35,
 34,
 20,
 22,
 62,
 34,
 56,
 32,
 50,
 80,
 51,
 52,
 48,
 47,
 25,
 48,
 34,
 37,
 38,
 52,
 44,
 33,
 49,
 60,
 52,
 65,
 47,
 40,
 38,
 44,
 52,
 43,
 44,
 33,
 37,
 44,
 45,
 45,
 41,
 28,
 32,
 37,
 22,
 22,
 41,
 58,
 64,
 51,
 33,
 49,
 68,
 51,
 50,
 42,
 41,
 31,
 60,
 69,


[46,
 21,
 49,
 58,
 38,
 56,
 27,
 40,
 45,
 31,
 53,
 59,
 25,
 39,
 51,
 47,
 26,
 46,
 43,
 39,
 40,
 70,
 24,
 32,
 50,
 67,
 44,
 37,
 52,
 56,
 32,
 40,
 52,
 50,
 58,
 62,
 47,
 22,
 47,
 26,
 38,
 63,
 35,
 41,
 30,
 50,
 35,
 42,
 69,
 28,
 30,
 69,
 41,
 24,
 72,
 48,
 31,
 31,
 48,
 39,
 78,
 38,
 58,
 59,
 33,
 31,
 31,
 71,
 22,
 66,
 67,
 41,
 32,
 55,
 33,
 55,
 44,
 66,
 60,
 41,
 35,
 46,
 32,
 24,
 53,
 50,
 29,
 45,
 33,
 27,
 26,
 26,
 34,
 57,
 26,
 28,
 51,
 46,
 38,
 42,
 40,
 22,
 53,
 56,
 52,
 52,
 58,
 61,
 25,
 40,
 45,
 57,
 22,
 24,
 36,
 37,
 48,
 33,
 54,
 55,
 48,
 39,
 34,
 56,
 41,
 29,
 46,
 34,
 55,
 45,
 53,
 32,
 54,
 40,
 41,
 52,
 56,
 34,
 35,
 36,
 38,
 52,
 35,
 34,
 20,
 22,
 62,
 34,
 56,
 32,
 50,
 80,
 51,
 52,
 48,
 47,
 25,
 48,
 34,
 37,
 38,
 52,
 44,
 33,
 49,
 60,
 52,
 65,
 47,
 40,
 38,
 44,
 52,
 43,
 44,
 33,
 37,
 44,
 45,
 45,
 41,
 28,
 32,
 37,
 22,
 22,
 41,
 58,
 64,
 51,
 33,
 49,
 68,
 51,
 50,
 42,
 41,
 31,
 60,
 69,


## converting lists into data frames and concat 

In [144]:
driver_age_ml = pd.DataFrame({'driver_age':driver_age_clean_ml})
driver_age_bi = pd.DataFrame({'driver_age':driver_age_clean_bi})
salary_bi_clean = pd.DataFrame({'salary':salary_clean_bi})
salary_ml_clean = pd.DataFrame({'salary':salary_clean_ml})


In [145]:
#concatenation
claim_bi_df = pd.concat([claim_bi_df,driver_age_bi,salary_bi_clean], axis = 1)
claim_ml_df = pd.concat([claim_ml_df,driver_age_ml,salary_ml_clean], axis = 1)

## validate

In [146]:
def head_data(data):
    head = data.head()
    return head

display(head_data(claim_bi_df))
display(head_data(claim_ml_df))

,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,46,M,1.0,85,38301,1,1,Rent,80006,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,21,F,0.0,75,30445,0,1,Rent,15021,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,49,F,0.0,87,38923,0,1,Own,20158,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,58,F,1.0,58,40605,1,0,Own,15024,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,38,M,1.0,95,36380,1,0,Rent,50034,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0


,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,46,M,1.0,85,38301,1,1,Rent,80006,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,21,F,0.0,75,30445,0,1,Rent,15021,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,49,F,0.0,87,38923,0,1,Own,20158,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,58,F,1.0,58,40605,1,0,Own,15024,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,38,M,1.0,95,36380,1,0,Rent,50034,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0


## drop original columns

In [147]:
claim_bi_df = claim_bi_df.drop(columns = {'age_of_driver', 'annual_income'}, axis = 1)
claim_ml_df = claim_ml_df.drop(columns = {'age_of_driver', 'annual_income'}, axis = 1)

In [148]:
claim_ml_df

,claim_number,gender,marital_status,safty_rating,high_education_ind,address_change_ind,living_status,zip_code,claim_date,claim_day_of_week,...,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,29993,M,1.0,93,1,0,Own,15012,2015-03-28,Saturday,...,0,4807.0,6.0,Medium,32145.0,black,16868.53,0,69,42338.0
17994,29996,F,0.0,22,1,1,Own,20111,2015-01-05,Monday,...,1,4090.0,3.0,Medium,9469.0,blue,31501.00,1,35,35579.0
17995,29997,F,1.0,81,0,1,Rent,15012,2016-09-10,Saturday,...,1,2226.0,5.0,Medium,64974.0,black,44536.26,0,27,32953.0
17996,29999,F,1.0,86,1,0,Own,15026,2015-12-25,Friday,...,1,9561.0,3.0,Compact,24983.0,white,50093.76,0,52,39519.0


In [149]:
claim_bi_df

,claim_number,gender,marital_status,safty_rating,high_education_ind,address_change_ind,living_status,zip_code,claim_date,claim_day_of_week,...,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,29993,M,1.0,93,1,0,Own,15012,2015-03-28,Saturday,...,0,4807.0,6.0,Medium,32145.0,black,16868.53,0,69,42338.0
17994,29996,F,0.0,22,1,1,Own,20111,2015-01-05,Monday,...,1,4090.0,3.0,Medium,9469.0,blue,31501.00,1,35,35579.0
17995,29997,F,1.0,81,0,1,Rent,15012,2016-09-10,Saturday,...,1,2226.0,5.0,Medium,64974.0,black,44536.26,0,27,32953.0
17996,29999,F,1.0,86,1,0,Own,15026,2015-12-25,Friday,...,1,9561.0,3.0,Compact,24983.0,white,50093.76,0,52,39519.0


## checking for duplicate claim numbers

In [150]:
#returns true for duplicated, false if not duplicated 
duplicate = claim_bi_df['claim_number'].duplicated()

print('duplicate count: ', duplicate[duplicate == True].count())
print('non-duplicate count: ', duplicate[duplicate == False].count())

duplicate count:  0
non-duplicate count:  17998


## check the nulls 

In [151]:
#null values
print('ML dataset nulls: \n')
display(claim_ml_df.isna().sum())
print('powerBI dataset nulls: \n')
display(claim_bi_df.isna().sum())

ML dataset nulls: 



claim_number               0
gender                     0
marital_status             0
safty_rating               0
high_education_ind         0
address_change_ind         0
living_status              0
zip_code                   0
claim_date                 0
claim_day_of_week          0
accident_site              0
past_num_of_claims         0
witness_present_ind        0
liab_prct                  0
channel                    0
policy_report_filed_ind    0
claim_est_payout           0
age_of_vehicle             0
vehicle_category           0
vehicle_price              0
vehicle_color              0
vehicle_weight             0
fraud                      0
driver_age                 0
salary                     0
dtype: int64

powerBI dataset nulls: 



claim_number               0
gender                     0
marital_status             0
safty_rating               0
high_education_ind         0
address_change_ind         0
living_status              0
zip_code                   0
claim_date                 0
claim_day_of_week          0
accident_site              0
past_num_of_claims         0
witness_present_ind        0
liab_prct                  0
channel                    0
policy_report_filed_ind    0
claim_est_payout           0
age_of_vehicle             0
vehicle_category           0
vehicle_price              0
vehicle_color              0
vehicle_weight             0
fraud                      0
driver_age                 0
salary                     0
dtype: int64

## Renaming columns

In [152]:
claim_ml_df = claim_ml_df.rename(columns = {'safty_rating':'safety_rating',
                                           'high_education_ind':'college_graduate',
                                           'address_change_ind':'address_change',
                                           'past_num_of_claims':'no._of_claims',
                                           'liab_prct':'liab(%)',
                                           'claim_est_payout':'claim_payout',
                                           'age_of_vehicle':'vehicle_age',
                                           'witness_present_ind':'witness_present'})
  
claim_bi_df = claim_bi_df.rename(columns = {'safty_rating':'safety_rating',
                                           'high_education_ind':'college_graduate',
                                           'address_change_ind':'address_change',
                                           'past_num_of_claims':'no._of_claims',
                                           'liab_prct':'liab(%)',
                                           'claim_est_payout':'claim_payout',
                                           'age_of_vehicle':'vehicle_age',
                                           'witness_present_ind':'witness_present'})
                                            


## validate

In [153]:
head_data(claim_bi_df)


,claim_number,gender,marital_status,safety_rating,college_graduate,address_change,living_status,zip_code,claim_date,claim_day_of_week,...,policy_report_filed_ind,claim_payout,vehicle_age,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0


In [154]:
head_data(claim_ml_df)

,claim_number,gender,marital_status,safety_rating,college_graduate,address_change,living_status,zip_code,claim_date,claim_day_of_week,...,policy_report_filed_ind,claim_payout,vehicle_age,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0


## Data Cleaning: Category mapping

In [155]:
#spark dataframes
claim_ml_sp = spark.createDataFrame(claim_ml_df)
claim_bi_sp = spark.createDataFrame(claim_bi_df)

## age ranges

In [156]:
#machine learning data 
claim_ml_sp = claim_ml_sp.withColumn('age_range', when((claim_ml_sp['driver_age'] >= 17) & (claim_ml_sp['driver_age'] < 26), ('17 to 25')).otherwise(claim_ml_sp['driver_age']))
                                           
claim_ml_sp = claim_ml_sp.withColumn('age_range', when((claim_ml_sp['age_range'] >= 26) & (claim_ml_sp['age_range'] < 35), ('26 to 34')).otherwise(claim_ml_sp['age_range']))
 
claim_ml_sp = claim_ml_sp.withColumn('age_range', when((claim_ml_sp['age_range'] >= 35) & (claim_ml_sp['age_range'] < 44), ('35 to 43')).otherwise(claim_ml_sp['age_range']))
                                           
claim_ml_sp = claim_ml_sp.withColumn('age_range', when((claim_ml_sp['age_range'] >= 44) & (claim_ml_sp['age_range'] < 53), ('44 to 52')).otherwise(claim_ml_sp['age_range']))

claim_ml_sp = claim_ml_sp.withColumn('age_range', when((claim_ml_sp['age_range'] >= 53) & (claim_ml_sp['age_range'] < 62), ('53 to 61')).otherwise(claim_ml_sp['age_range']))

claim_ml_sp = claim_ml_sp.withColumn('age_range', when((claim_ml_sp['age_range'] >= 62), '63 and over').otherwise(claim_ml_sp['age_range']))

#powerbi data 
claim_bi_sp = claim_bi_sp.withColumn('age_range', when((claim_bi_sp['driver_age'] >= 17) & (claim_bi_sp['driver_age'] < 26), ('17 to 25')).otherwise(claim_bi_sp['driver_age']))
                                              
claim_bi_sp = claim_bi_sp.withColumn('age_range', when((claim_bi_sp['age_range'] >= 26) & (claim_bi_sp['age_range'] < 35), ('26 to 34')).otherwise(claim_bi_sp['age_range']))
                                           
claim_bi_sp = claim_bi_sp.withColumn('age_range', when((claim_bi_sp['age_range'] >= 35) & (claim_bi_sp['age_range'] < 44), ('35 to 43')).otherwise(claim_bi_sp['age_range']))
                                                                            
claim_bi_sp = claim_bi_sp.withColumn('age_range', when((claim_bi_sp['age_range'] >= 44) & (claim_bi_sp['age_range'] < 53), ('44 to 52')).otherwise(claim_bi_sp['age_range']))
 
claim_bi_sp = claim_bi_sp.withColumn('age_range', when((claim_bi_sp['age_range'] >= 53) & (claim_bi_sp['age_range'] < 62), ('53 to 61')).otherwise(claim_bi_sp['age_range']))
    
claim_bi_sp = claim_bi_sp.withColumn('age_range', when((claim_bi_sp['age_range'] >= 62), '62 and over').otherwise(claim_bi_sp['age_range']))
                                           
    
                                           
                                           
                                           
                                           
                                           
                                           
                                           
                                           
                                           


In [157]:
claim_ml_sp.select('age_range').show()

+---------+
|age_range|
+---------+
| 44 to 52|
| 17 to 25|
| 44 to 52|
| 53 to 61|
| 35 to 43|
| 53 to 61|
| 26 to 34|
| 35 to 43|
| 44 to 52|
| 26 to 34|
| 53 to 61|
| 53 to 61|
| 17 to 25|
| 35 to 43|
| 44 to 52|
| 44 to 52|
| 26 to 34|
| 44 to 52|
| 35 to 43|
| 35 to 43|
+---------+
only showing top 20 rows



In [158]:
claim_bi_sp.select('age_range').show()

+---------+
|age_range|
+---------+
| 44 to 52|
| 17 to 25|
| 44 to 52|
| 53 to 61|
| 35 to 43|
| 53 to 61|
| 26 to 34|
| 35 to 43|
| 44 to 52|
| 26 to 34|
| 53 to 61|
| 53 to 61|
| 17 to 25|
| 35 to 43|
| 44 to 52|
| 44 to 52|
| 26 to 34|
| 44 to 52|
| 35 to 43|
| 35 to 43|
+---------+
only showing top 20 rows



In [159]:
# age_range_bi = []
# age_range_ml = []

# for age1 in claim_ml_df['age_of_driver']:
#     if (age1 >= 17) & (age1 <= 25):
#         age_range_ml.append('17 to 25')
#     elif (age1 >= 26) & (age1 <= 34):
#         age_range_ml.append('26 to 35')
#     elif (age1 >= 36) & (age1 <= 48):
#         age_range_ml.append('36 to 48')
#     elif (age1 >= 49) & (age1 <= 57):
#         age_range_ml.append('49 to 57')
#     else:
#         age_range_ml.append('58+')

        
# for age2 in claim_bi_df['age_of_driver']:
#     if (age2 >= 17) & (age2 <= 25):
#         age_range_bi.append('17 to 25')
#     elif (age2 >= 26) & (age2 <= 34):
#         age_range_bi.append('26 to 35')
#     elif (age2 >= 36) & (age2 <= 48):
#         age_range_bi.append('36 to 48')
#     elif (age2 >= 49) & (age2 <= 57):
#         age_range_bi.append('49 to 57')
#     else:
#         age_range_bi.append('58+')
       
        
        
# display(age_range_bi)
# display(age_range_ml)

## Salary ranges 

In [160]:
#ML data 
claim_ml_sp = claim_ml_sp.withColumn('salary_range', when((claim_ml_sp['salary'] <= 21000.0), ('21000 or less')).otherwise(claim_ml_sp['salary']))
                                           
claim_ml_sp = claim_ml_sp.withColumn('salary_range', when((claim_ml_sp['salary_range'] >= 21001.0) & (claim_ml_sp['salary_range'] <= 31000.0), ('21001 to 31000')).otherwise(claim_ml_sp['salary_range']))
 
claim_ml_sp = claim_ml_sp.withColumn('salary_range', when((claim_ml_sp['salary_range'] >= 31001.0) & (claim_ml_sp['salary_range'] <= 40000.0), ('31001 to 40000')).otherwise(claim_ml_sp['salary_range']))
   
claim_ml_sp = claim_ml_sp.withColumn('salary_range', when((claim_ml_sp['salary_range'] >= 40001.0) & (claim_ml_sp['salary_range'] <= 50000.0), ('40001 to 50000')).otherwise(claim_ml_sp['salary_range']))

claim_ml_sp = claim_ml_sp.withColumn('salary_range', when((claim_ml_sp['salary_range'] >= 50001.0), ('50001+')).otherwise(claim_ml_sp['salary_range']))


#powerbi
claim_bi_sp = claim_bi_sp.withColumn('salary_range', when((claim_bi_sp['salary'] <= 21000.0), ('21000 or less')).otherwise(claim_bi_sp['salary']))

claim_bi_sp = claim_bi_sp.withColumn('salary_range', when((claim_bi_sp['salary_range'] >= 21001.0) & (claim_bi_sp['salary_range'] <= 31000.0), ('21001 to 31000')).otherwise(claim_bi_sp['salary_range']))
 
claim_bi_sp = claim_bi_sp.withColumn('salary_range', when((claim_bi_sp['salary_range'] >= 31001.0) & (claim_bi_sp['salary_range'] <= 40000.0), ('31001 to 40000')).otherwise(claim_bi_sp['salary_range']))
   
claim_bi_sp = claim_bi_sp.withColumn('salary_range', when((claim_bi_sp['salary_range'] >= 40001.0) & (claim_bi_sp['salary_range'] <= 50000.0), ('40001 to 50000')).otherwise(claim_bi_sp['salary_range']))

claim_bi_sp = claim_bi_sp.withColumn('salary_range', when((claim_bi_sp['salary_range'] >= 50001.0), ('50001+')).otherwise(claim_bi_sp['salary_range']))


In [161]:
claim_bi_df.head()

,claim_number,gender,marital_status,safety_rating,college_graduate,address_change,living_status,zip_code,claim_date,claim_day_of_week,...,policy_report_filed_ind,claim_payout,vehicle_age,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0


In [162]:
claim_bi_sp.select('salary_range').show()

+--------------+
|  salary_range|
+--------------+
|31001 to 40000|
|21001 to 31000|
|31001 to 40000|
|40001 to 50000|
|31001 to 40000|
|40001 to 50000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
|40001 to 50000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
|31001 to 40000|
+--------------+
only showing top 20 rows



In [163]:
claim_bi_sp.select('claim_payout').where(claim_bi_sp['claim_payout'] < 1000.0).show()

+------------+
|claim_payout|
+------------+
|       709.0|
|       746.0|
|       819.0|
|       984.0|
|       894.0|
|       509.0|
|       760.0|
|       540.0|
|       974.0|
|       830.0|
|       949.0|
|       803.0|
|       966.0|
|       787.0|
|       993.0|
|       934.0|
|       283.0|
|       861.0|
|       549.0|
|       993.0|
+------------+
only showing top 20 rows



## claim payout 

In [164]:
#ML data 
claim_ml_sp = claim_ml_sp.withColumn('compensation_payout', 
                                     when((claim_ml_sp['claim_payout'] <= 1000.0), 
                                          ('1000 or less')).otherwise(claim_ml_sp['claim_payout']))
                                           
claim_ml_sp = claim_ml_sp.withColumn('compensation_payout', 
                                     when((claim_ml_sp['compensation_payout'] >= 1001.0) & (claim_ml_sp['compensation_payout'] <= 5000.0),
                                          ('1001 to 5000')).otherwise(claim_ml_sp['compensation_payout']))
 
claim_ml_sp = claim_ml_sp.withColumn('compensation_payout', 
                                     when((claim_ml_sp['compensation_payout'] >= 4001.0) & (claim_ml_sp['compensation_payout'] <= 8000.0),
                                          ('4001 to 8000')).otherwise(claim_ml_sp['compensation_payout']))
   
claim_ml_sp = claim_ml_sp.withColumn('compensation_payout', 
                                     when((claim_ml_sp['compensation_payout'] >= 8001.0) & (claim_ml_sp['compensation_payout'] <= 12000.0),
                                          ('8001 to 12000')).otherwise(claim_ml_sp['compensation_payout']))

claim_ml_sp = claim_ml_sp.withColumn('compensation_payout', when((claim_ml_sp['compensation_payout'] >= 12001.0), 
                                                                 ('12001+')).otherwise(claim_ml_sp['compensation_payout']))


In [165]:
claim_ml_sp.select('compensation_payout').show()

+-------------------+
|compensation_payout|
+-------------------+
|       4001 to 8000|
|       1001 to 5000|
|       4001 to 8000|
|       4001 to 8000|
|       1001 to 5000|
|       4001 to 8000|
|      8001 to 12000|
|             12001+|
|       4001 to 8000|
|       4001 to 8000|
|      8001 to 12000|
|       4001 to 8000|
|       1001 to 5000|
|       1001 to 5000|
|       4001 to 8000|
|       4001 to 8000|
|      8001 to 12000|
|       1001 to 5000|
|       4001 to 8000|
|       4001 to 8000|
+-------------------+
only showing top 20 rows



In [166]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('compensation_payout', when((claim_bi_sp['claim_payout'] <= 1000.0), ('1000 or less')).otherwise(claim_bi_sp['claim_payout']))
                                           
claim_bi_sp = claim_bi_sp.withColumn('compensation_payout', when((claim_bi_sp['compensation_payout'] >= 1001.0) & (claim_bi_sp['compensation_payout'] <= 5000.0), ('1001 to 5000')).otherwise(claim_bi_sp['compensation_payout']))
 
claim_bi_sp = claim_bi_sp.withColumn('compensation_payout', when((claim_bi_sp['compensation_payout'] >= 4001.0) & (claim_bi_sp['compensation_payout'] <= 8000.0), ('4001 to 8000')).otherwise(claim_bi_sp['compensation_payout']))
   
claim_bi_sp = claim_bi_sp.withColumn('compensation_payout', when((claim_bi_sp['compensation_payout'] >= 8001.0) & (claim_bi_sp['compensation_payout'] <= 12000.0), ('8001 to 12000')).otherwise(claim_bi_sp['compensation_payout']))

claim_bi_sp = claim_bi_sp.withColumn('compensation_payout', when((claim_bi_sp['compensation_payout'] >= 12001.0), ('12001+')).otherwise(claim_bi_sp['compensation_payout']))


In [167]:
claim_bi_sp.select('compensation_payout').show()

+-------------------+
|compensation_payout|
+-------------------+
|       4001 to 8000|
|       1001 to 5000|
|       4001 to 8000|
|       4001 to 8000|
|       1001 to 5000|
|       4001 to 8000|
|      8001 to 12000|
|             12001+|
|       4001 to 8000|
|       4001 to 8000|
|      8001 to 12000|
|       4001 to 8000|
|       1001 to 5000|
|       1001 to 5000|
|       4001 to 8000|
|       4001 to 8000|
|      8001 to 12000|
|       1001 to 5000|
|       4001 to 8000|
|       4001 to 8000|
+-------------------+
only showing top 20 rows



## vehicle price 

In [168]:
claim_ml_df['vehicle_price'].max()
claim_ml_df['vehicle_price'].min()

2457.0

In [169]:
#ML data
claim_ml_sp = claim_ml_sp.withColumn('vehicle_price_range', 
                                     when((claim_ml_sp['vehicle_price'] <= 5000.0), 
                                          ('5000 or less')).otherwise(claim_ml_sp['vehicle_price']))
                                           
claim_ml_sp = claim_ml_sp.withColumn('vehicle_price_range', 
                                     when((claim_ml_sp['vehicle_price_range'] >= 5001.0) & (claim_ml_sp['vehicle_price_range'] <= 20000.0),
                                          ('5001 to 20000')).otherwise(claim_ml_sp['vehicle_price_range']))
 
claim_ml_sp = claim_ml_sp.withColumn('vehicle_price_range', 
                                     when((claim_ml_sp['vehicle_price_range'] >= 15001.0) & (claim_ml_sp['vehicle_price_range'] <= 30000.0),
                                          ('15001 to 30000')).otherwise(claim_ml_sp['vehicle_price_range']))
   
claim_ml_sp = claim_ml_sp.withColumn('vehicle_price_range', 
                                     when((claim_ml_sp['vehicle_price_range'] >= 30001.0) & (claim_ml_sp['vehicle_price_range'] <= 45000.0),
                                          ('30001 to 45000')).otherwise(claim_ml_sp['vehicle_price_range']))

claim_ml_sp = claim_ml_sp.withColumn('vehicle_price_range', 
                                     when((claim_ml_sp['vehicle_price_range'] >= 45001.0) & (claim_ml_sp['vehicle_price_range'] <= 60000.0), 
                                                                 ('45001 to 60000')).otherwise(claim_ml_sp['vehicle_price_range']))

claim_ml_sp = claim_ml_sp.withColumn('vehicle_price_range', 
                                     when((claim_ml_sp['vehicle_price_range'] >= 60001.0), 
                                                                 ('60000+')).otherwise(claim_ml_sp['vehicle_price_range']))








In [170]:
#BI data
claim_bi_sp = claim_bi_sp.withColumn('vehicle_price_range', 
                                     when((claim_bi_sp['vehicle_price'] <= 5000.0), 
                                          ('5000 or less')).otherwise(claim_bi_sp['vehicle_price']))
                                           
claim_bi_sp = claim_bi_sp.withColumn('vehicle_price_range', 
                                     when((claim_bi_sp['vehicle_price_range'] >= 5001.0) & (claim_bi_sp['vehicle_price_range'] <= 20000.0),
                                          ('5001 to 20000')).otherwise(claim_bi_sp['vehicle_price_range']))
 
claim_bi_sp = claim_bi_sp.withColumn('vehicle_price_range', 
                                     when((claim_bi_sp['vehicle_price_range'] >= 15001.0) & (claim_bi_sp['vehicle_price_range'] <= 30000.0),
                                          ('15001 to 30000')).otherwise(claim_bi_sp['vehicle_price_range']))
   
claim_bi_sp = claim_bi_sp.withColumn('vehicle_price_range', 
                                     when((claim_bi_sp['vehicle_price_range'] >= 30001.0) & (claim_bi_sp['vehicle_price_range'] <= 45000.0),
                                          ('30001 to 45000')).otherwise(claim_bi_sp['vehicle_price_range']))

claim_bi_sp = claim_bi_sp.withColumn('vehicle_price_range', 
                                     when((claim_bi_sp['vehicle_price_range'] >= 45001.0) & (claim_bi_sp['vehicle_price_range'] <= 60000.0), 
                                                                 ('45001 to 60000')).otherwise(claim_bi_sp['vehicle_price_range']))

claim_bi_sp = claim_bi_sp.withColumn('vehicle_price_range', 
                                     when((claim_bi_sp['vehicle_price_range'] >= 60001.0), 
                                                                 ('60000+')).otherwise(claim_bi_sp['vehicle_price_range']))



In [171]:
claim_bi_df.head(20)

,claim_number,gender,marital_status,safety_rating,college_graduate,address_change,living_status,zip_code,claim_date,claim_day_of_week,...,policy_report_filed_ind,claim_payout,vehicle_age,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0
5,7,F,0.0,72,0,0,Own,50047,2016-11-16,Wednesday,...,0,5389.0,4.0,Medium,17688.0,black,15234.08,1,56,40240.0
6,8,M,1.0,80,1,0,Rent,15001,2015-03-14,Saturday,...,1,8960.0,7.0,Compact,26412.0,red,28994.54,0,27,32952.0
7,10,M,1.0,79,1,1,Own,80016,2016-01-07,Thursday,...,1,13844.0,8.0,Medium,11355.0,other,34045.83,0,40,36891.0
8,12,F,0.0,86,0,1,Own,15024,2015-03-31,Tuesday,...,0,6039.0,5.0,Compact,49268.0,black,23206.51,0,45,38069.0
9,15,M,1.0,80,0,1,Rent,85032,2015-09-03,Thursday,...,0,5870.0,5.0,Large,19930.0,white,37277.76,0,31,34324.0


## date features: day, month and year

In [172]:
#year
claim_bi_sp = claim_bi_sp.withColumn('year', year(claim_bi_sp['claim_date']))
claim_ml_sp = claim_ml_sp.withColumn('year', year(claim_ml_sp['claim_date']))

#month
claim_bi_sp = claim_bi_sp.withColumn('month_coded', month(claim_bi_sp['claim_date']))
claim_ml_sp = claim_ml_sp.withColumn('month_coded', month(claim_ml_sp['claim_date']))

#day
claim_bi_sp = claim_bi_sp.withColumn('day_coded', dayofweek(claim_bi_sp['claim_date']))
claim_ml_sp = claim_ml_sp.withColumn('day_coded', dayofweek(claim_ml_sp['claim_date']))

## Validate

In [173]:
claim_bi_sp.select('year', 'month_coded', 'day_coded').show()
claim_ml_sp.select('year', 'month_coded', 'day_coded').show()

+----+-----------+---------+
|year|month_coded|day_coded|
+----+-----------+---------+
|2016|         12|        6|
|2015|          2|        5|
|2016|         12|        3|
|2016|          5|        5|
|2015|         10|        3|
|2016|         11|        4|
|2015|          3|        7|
|2016|          1|        5|
|2015|          3|        3|
|2015|          9|        5|
|2016|          5|        7|
|2015|          2|        5|
|2015|          7|        7|
|2015|          4|        2|
|2015|          6|        3|
|2015|          8|        4|
|2015|         11|        4|
|2016|          2|        6|
|2016|          4|        7|
|2015|         11|        3|
+----+-----------+---------+
only showing top 20 rows

+----+-----------+---------+
|year|month_coded|day_coded|
+----+-----------+---------+
|2016|         12|        6|
|2015|          2|        5|
|2016|         12|        3|
|2016|          5|        5|
|2015|         10|        3|
|2016|         11|        4|
|2015|          3

## reassigning month

In [174]:
#BI data
claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month_coded'] == 1), 
                                                   ('jan')).otherwise(claim_bi_sp['month_coded']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 2), 
                                                   ('feb')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 3), 
                                                   ('mar')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 4), 
                                                   ('apr')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 5), 
                                                   ('may')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 6), 
                                                   ('jun')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 7), 
                                                   ('jul')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 8), 
                                                   ('aug')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 9), 
                                                   ('sep')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 10), 
                                                   ('oct')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 11), 
                                                   ('nov')).otherwise(claim_bi_sp['month']))

claim_bi_sp = claim_bi_sp.withColumn('month', when((claim_bi_sp['month'] == 12), 
                                                   ('dec')).otherwise(claim_bi_sp['month']))






In [175]:
#ml data 

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month_coded'] == 1), 
                                                   ('jan')).otherwise(claim_ml_sp['month_coded']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 2), 
                                                   ('feb')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 3), 
                                                   ('mar')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 4), 
                                                   ('apr')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 5), 
                                                   ('may')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 6), 
                                                   ('jun')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 7), 
                                                   ('jul')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 8), 
                                                   ('aug')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 9), 
                                                   ('sep')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 10), 
                                                   ('oct')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 11), 
                                                   ('nov')).otherwise(claim_ml_sp['month']))

claim_ml_sp = claim_ml_sp.withColumn('month', when((claim_ml_sp['month'] == 12), 
                                                   ('dec')).otherwise(claim_ml_sp['month']))

# Reassigning data 

In [176]:
claim_bi_df.head()

,claim_number,gender,marital_status,safety_rating,college_graduate,address_change,living_status,zip_code,claim_date,claim_day_of_week,...,policy_report_filed_ind,claim_payout,vehicle_age,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0


## witness present 

In [177]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('witness_present_cat', 
                                     when((claim_bi_sp['witness_present'] == 0), 
                                          ('not present')).otherwise(claim_bi_sp['witness_present']))

claim_bi_sp = claim_bi_sp.withColumn('witness_present_cat', 
                                     when((claim_bi_sp['witness_present_cat'] == 1), 
                                          ('present')).otherwise(claim_bi_sp['witness_present_cat']))

In [178]:
#ml data 
claim_ml_sp = claim_ml_sp.withColumn('witness_present_cat', 
                                     when((claim_ml_sp['witness_present'] == 0), 
                                          ('not present')).otherwise(claim_ml_sp['witness_present']))

claim_ml_sp = claim_ml_sp.withColumn('witness_present_cat', 
                                     when((claim_ml_sp['witness_present_cat'] == 1), 
                                          ('present')).otherwise(claim_ml_sp['witness_present_cat']))

## marital status

In [179]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('marital_status_cat', 
                                     when((claim_bi_sp['marital_status'] == 0), 
                                          ('Not Married')).otherwise(claim_bi_sp['marital_status']))

claim_bi_sp = claim_bi_sp.withColumn('marital_status_cat', 
                                     when((claim_bi_sp['marital_status_cat'] == 1), 
                                          ('Married')).otherwise(claim_bi_sp['marital_status_cat']))

In [180]:
#ML data 
claim_ml_sp = claim_ml_sp.withColumn('marital_status_cat', 
                                     when((claim_ml_sp['marital_status'] == 0), 
                                          ('Not Married')).otherwise(claim_ml_sp['marital_status']))

claim_ml_sp = claim_ml_sp.withColumn('marital_status_cat', 
                                     when((claim_ml_sp['marital_status_cat'] == 1), 
                                          ('Married')).otherwise(claim_ml_sp['marital_status_cat']))


## education status

In [181]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('college_graduate_cat', 
                                     when((claim_bi_sp['college_graduate'] == 0), 
                                          ('not a college graduate')).otherwise(claim_bi_sp['college_graduate']))

claim_bi_sp = claim_bi_sp.withColumn('college_graduate_cat', 
                                     when((claim_bi_sp['college_graduate_cat'] == 1), 
                                          ('college graduate')).otherwise(claim_bi_sp['college_graduate_cat']))

In [182]:
#ML data 
claim_ml_sp = claim_ml_sp.withColumn('college_graduate_cat', 
                                     when((claim_ml_sp['college_graduate'] == 0), 
                                          ('not a college graduate')).otherwise(claim_ml_sp['college_graduate']))

claim_ml_sp = claim_ml_sp.withColumn('college_graduate_cat', 
                                     when((claim_ml_sp['college_graduate_cat'] == 1), 
                                          ('college graduate')).otherwise(claim_ml_sp['college_graduate_cat']))

In [183]:
claim_ml_sp.select('college_graduate_cat').show()

+--------------------+
|college_graduate_cat|
+--------------------+
|    college graduate|
|not a college gra...|
|not a college gra...|
|    college graduate|
|    college graduate|
|not a college gra...|
|    college graduate|
|    college graduate|
|not a college gra...|
|not a college gra...|
|    college graduate|
|    college graduate|
|    college graduate|
|not a college gra...|
|not a college gra...|
|not a college gra...|
|    college graduate|
|    college graduate|
|not a college gra...|
|    college graduate|
+--------------------+
only showing top 20 rows



In [184]:
claim_bi_sp.select('college_graduate_cat').show()

+--------------------+
|college_graduate_cat|
+--------------------+
|    college graduate|
|not a college gra...|
|not a college gra...|
|    college graduate|
|    college graduate|
|not a college gra...|
|    college graduate|
|    college graduate|
|not a college gra...|
|not a college gra...|
|    college graduate|
|    college graduate|
|    college graduate|
|not a college gra...|
|not a college gra...|
|not a college gra...|
|    college graduate|
|    college graduate|
|not a college gra...|
|    college graduate|
+--------------------+
only showing top 20 rows



## address change

In [185]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('address_change_cat', 
                                     when((claim_bi_sp['address_change'] == 0), 
                                          ('not changed')).otherwise(claim_bi_sp['address_change']))

claim_bi_sp = claim_bi_sp.withColumn('address_change_cat', 
                                     when((claim_bi_sp['address_change_cat'] == 1), 
                                          ('address changed')).otherwise(claim_bi_sp['address_change_cat']))

In [186]:
#ML data 
claim_ml_sp = claim_ml_sp.withColumn('address_change_cat', 
                                     when((claim_ml_sp['address_change'] == 0), 
                                          ('not changed')).otherwise(claim_ml_sp['address_change']))

claim_ml_sp = claim_ml_sp.withColumn('address_change_cat', 
                                     when((claim_ml_sp['address_change_cat'] == 1), 
                                          ('address changed')).otherwise(claim_ml_sp['address_change_cat']))

In [187]:
claim_bi_df.head()

,claim_number,gender,marital_status,safety_rating,college_graduate,address_change,living_status,zip_code,claim_date,claim_day_of_week,...,policy_report_filed_ind,claim_payout,vehicle_age,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,driver_age,salary
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,0,7531.0,9.0,Compact,12885.0,white,16161.33,0,46,38301.0
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,0,2966.0,4.0,Large,29429.0,white,28691.96,0,21,30445.0
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,0,6284.0,3.0,Compact,21701.0,white,22090.95,1,49,38923.0
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,1,6170.0,4.0,Medium,13198.0,other,38329.58,1,58,40605.0
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,0,4541.0,7.0,Medium,38060.0,gray,25876.56,0,38,36380.0


## safety rating

In [188]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('safety_rating_cat', 
                                     when((claim_bi_sp['safety_rating'] <= 20), 
                                          ('20 or less')).otherwise(claim_bi_sp['safety_rating']))

claim_bi_sp = claim_bi_sp.withColumn('safety_rating_cat', 
                                     when((claim_bi_sp['safety_rating_cat'] >= 21) & (claim_bi_sp['safety_rating_cat'] <= 40),
                                          ('21 to 40)')).otherwise(claim_bi_sp['safety_rating_cat']))

claim_bi_sp = claim_bi_sp.withColumn('safety_rating_cat', 
                                     when((claim_bi_sp['safety_rating_cat'] >= 41) & (claim_bi_sp['safety_rating_cat'] <= 60), 
                                          ('41 to 60')).otherwise(claim_bi_sp['safety_rating_cat']))

claim_bi_sp = claim_bi_sp.withColumn('safety_rating_cat', 
                                     when((claim_bi_sp['safety_rating_cat'] >= 61) & (claim_bi_sp['safety_rating_cat'] <= 80), 
                                          ('61 to 80')).otherwise(claim_bi_sp['safety_rating_cat']))

claim_bi_sp = claim_bi_sp.withColumn('safety_rating_cat', 
                                     when((claim_bi_sp['safety_rating_cat'] >= 81) & (claim_bi_sp['safety_rating_cat'] <= 100), 
                                          ('81 to 100')).otherwise(claim_bi_sp['safety_rating_cat']))


In [189]:
#ml data 
claim_ml_sp = claim_ml_sp.withColumn('safety_rating_cat', 
                                     when((claim_ml_sp['safety_rating'] <= 20), 
                                          ('20 or less')).otherwise(claim_ml_sp['safety_rating']))

claim_ml_sp = claim_ml_sp.withColumn('safety_rating_cat', 
                                     when((claim_ml_sp['safety_rating_cat'] >= 21) & (claim_ml_sp['safety_rating_cat'] <= 40),
                                          ('21 to 40')).otherwise(claim_ml_sp['safety_rating_cat']))

claim_ml_sp = claim_ml_sp.withColumn('safety_rating_cat', 
                                     when((claim_ml_sp['safety_rating_cat'] >= 41) & (claim_ml_sp['safety_rating_cat'] <= 60), 
                                          ('41 to 60')).otherwise(claim_ml_sp['safety_rating_cat']))

claim_ml_sp = claim_ml_sp.withColumn('safety_rating_cat', 
                                     when((claim_ml_sp['safety_rating_cat'] >= 61) & (claim_ml_sp['safety_rating_cat'] <= 80), 
                                          ('61 to 80')).otherwise(claim_ml_sp['safety_rating_cat']))

claim_ml_sp = claim_ml_sp.withColumn('safety_rating_cat', 
                                     when((claim_ml_sp['safety_rating_cat'] >= 81) & (claim_ml_sp['safety_rating_cat'] <= 100), 
                                          ('81 to 100')).otherwise(claim_ml_sp['safety_rating_cat']))


## liab %

In [190]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('liab(%)_cat', 
                                     when((claim_bi_sp['liab(%)'] <= 20), 
                                          ('20 or less')).otherwise(claim_bi_sp['liab(%)']))

claim_bi_sp = claim_bi_sp.withColumn('liab(%)_cat', 
                                     when((claim_bi_sp['liab(%)_cat'] >= 21) & (claim_bi_sp['liab(%)_cat'] <= 40),
                                          ('21 to 40')).otherwise(claim_bi_sp['liab(%)_cat']))

claim_bi_sp = claim_bi_sp.withColumn('liab(%)_cat', 
                                     when((claim_bi_sp['liab(%)_cat'] >= 41) & (claim_bi_sp['liab(%)_cat'] <= 60), 
                                          ('41 to 60')).otherwise(claim_bi_sp['liab(%)_cat']))

claim_bi_sp = claim_bi_sp.withColumn('liab(%)_cat', 
                                     when((claim_bi_sp['liab(%)_cat'] >= 61) & (claim_bi_sp['liab(%)_cat'] <= 80), 
                                          ('61 to 80')).otherwise(claim_bi_sp['liab(%)_cat']))

claim_bi_sp = claim_bi_sp.withColumn('liab(%)_cat', 
                                     when((claim_bi_sp['liab(%)_cat'] >= 81) & (claim_bi_sp['liab(%)_cat'] <= 100), 
                                          ('81 to 100')).otherwise(claim_bi_sp['liab(%)_cat']))


In [191]:
#ML data 
claim_ml_sp = claim_ml_sp.withColumn('liab(%)_cat', 
                                     when((claim_ml_sp['liab(%)'] <= 20), 
                                          ('20 or less')).otherwise(claim_ml_sp['liab(%)']))

claim_ml_sp = claim_ml_sp.withColumn('liab(%)_cat', 
                                     when((claim_ml_sp['liab(%)_cat'] >= 21) & (claim_ml_sp['liab(%)_cat'] <= 40),
                                          ('21 to 40')).otherwise(claim_ml_sp['liab(%)_cat']))

claim_ml_sp = claim_ml_sp.withColumn('liab(%)_cat', 
                                     when((claim_ml_sp['liab(%)_cat'] >= 41) & (claim_ml_sp['liab(%)_cat'] <= 60), 
                                          ('41 to 60')).otherwise(claim_ml_sp['liab(%)_cat']))

claim_ml_sp = claim_ml_sp.withColumn('liab(%)_cat', 
                                     when((claim_ml_sp['liab(%)_cat'] >= 61) & (claim_ml_sp['liab(%)_cat'] <= 80), 
                                          ('61 to 80')).otherwise(claim_ml_sp['liab(%)_cat']))

claim_ml_sp = claim_ml_sp.withColumn('liab(%)_cat', 
                                     when((claim_ml_sp['liab(%)_cat'] >= 81) & (claim_ml_sp['liab(%)_cat'] <= 100), 
                                          ('81 to 100')).otherwise(claim_ml_sp['liab(%)_cat']))

## policy filled in 

In [192]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('policy_report', 
                                     when((claim_bi_sp['policy_report_filed_ind'] == 0), 
                                          ('incomplete')).otherwise(claim_bi_sp['policy_report_filed_ind']))

claim_bi_sp = claim_bi_sp.withColumn('policy_report', 
                                     when((claim_bi_sp['policy_report'] == 1), 
                                          ('complete')).otherwise(claim_bi_sp['policy_report']))

In [193]:
#ml data 
claim_ml_sp = claim_ml_sp.withColumn('policy_report', 
                                     when((claim_ml_sp['policy_report_filed_ind'] == 0), 
                                          ('incomplete')).otherwise(claim_ml_sp['policy_report_filed_ind']))

claim_ml_sp = claim_ml_sp.withColumn('policy_report', 
                                     when((claim_ml_sp['policy_report'] == 1), 
                                          ('complete')).otherwise(claim_ml_sp['policy_report']))

# fraud

In [194]:
#BI data 
claim_bi_sp = claim_bi_sp.withColumn('fraud_or_not', 
                                     when((claim_bi_sp['fraud'] == 0), 
                                          ('legitimate')).otherwise(claim_bi_sp['fraud']))

claim_bi_sp = claim_bi_sp.withColumn('fraud_or_not', 
                                     when((claim_bi_sp['fraud_or_not'] == 1), 
                                          ('fraudulent')).otherwise(claim_bi_sp['fraud_or_not']))

In [195]:
#ML data 
claim_ml_sp = claim_ml_sp.withColumn('fraud_or_not', 
                                     when((claim_ml_sp['fraud'] == 0), 
                                          ('legitimate')).otherwise(claim_ml_sp['fraud']))

claim_ml_sp = claim_ml_sp.withColumn('fraud_or_not', 
                                     when((claim_ml_sp['fraud_or_not'] == 1), 
                                          ('fraudulent')).otherwise(claim_ml_sp['fraud_or_not']))

In [196]:
claim_bi_sp.select('fraud_or_not').show()

+------------+
|fraud_or_not|
+------------+
|  legitimate|
|  legitimate|
|  fraudulent|
|  fraudulent|
|  legitimate|
|  fraudulent|
|  legitimate|
|  legitimate|
|  legitimate|
|  legitimate|
|  legitimate|
|  legitimate|
|  legitimate|
|  fraudulent|
|  legitimate|
|  fraudulent|
|  legitimate|
|  fraudulent|
|  fraudulent|
|  legitimate|
+------------+
only showing top 20 rows



In [197]:
claim_ml_sp.select('fraud_or_not').show()

+------------+
|fraud_or_not|
+------------+
|  legitimate|
|  legitimate|
|  fraudulent|
|  fraudulent|
|  legitimate|
|  fraudulent|
|  legitimate|
|  legitimate|
|  legitimate|
|  legitimate|
|  legitimate|
|  legitimate|
|  legitimate|
|  fraudulent|
|  legitimate|
|  fraudulent|
|  legitimate|
|  fraudulent|
|  fraudulent|
|  legitimate|
+------------+
only showing top 20 rows



# converting to pandas 

In [198]:
claim_bi_df = claim_bi_sp.toPandas()
claim_ml_df = claim_ml_sp.toPandas()


## saving csv for powerBI 

## dropping columns 

In [199]:
claim_bi_df.columns

Index(['claim_number', 'gender', 'marital_status', 'safety_rating',
       'college_graduate', 'address_change', 'living_status', 'zip_code',
       'claim_date', 'claim_day_of_week', 'accident_site', 'no._of_claims',
       'witness_present', 'liab(%)', 'channel', 'policy_report_filed_ind',
       'claim_payout', 'vehicle_age', 'vehicle_category', 'vehicle_price',
       'vehicle_color', 'vehicle_weight', 'fraud', 'driver_age', 'salary',
       'age_range', 'salary_range', 'compensation_payout',
       'vehicle_price_range', 'year', 'month_coded', 'day_coded', 'month',
       'witness_present_cat', 'marital_status_cat', 'college_graduate_cat',
       'address_change_cat', 'safety_rating_cat', 'liab(%)_cat',
       'policy_report', 'fraud_or_not'],
      dtype='object')

In [200]:
claim_bi_df.head()

,claim_number,gender,marital_status,safety_rating,college_graduate,address_change,living_status,zip_code,claim_date,claim_day_of_week,...,day_coded,month,witness_present_cat,marital_status_cat,college_graduate_cat,address_change_cat,safety_rating_cat,liab(%)_cat,policy_report,fraud_or_not
0,1,M,1.0,85,1,1,Rent,80006,2016-12-16,Friday,...,6,dec,not present,Married,college graduate,address changed,81 to 100,61 to 80,incomplete,legitimate
1,3,F,0.0,75,0,1,Rent,15021,2015-02-12,Thursday,...,5,feb,present,Not Married,not a college graduate,address changed,61 to 80,61 to 80,incomplete,legitimate
2,4,F,0.0,87,0,1,Own,20158,2016-12-06,Tuesday,...,3,dec,not present,Not Married,not a college graduate,address changed,81 to 100,20 or less,incomplete,fraudulent
3,5,F,1.0,58,1,0,Own,15024,2016-05-05,Thursday,...,5,may,not present,Married,college graduate,not changed,41 to 60,81 to 100,complete,fraudulent
4,6,M,1.0,95,1,0,Rent,50034,2015-10-27,Tuesday,...,3,oct,present,Married,college graduate,not changed,81 to 100,20 or less,incomplete,legitimate


In [201]:
claim_bi_df = claim_bi_df.drop(['marital_status',
                               'college_graduate',
                               'address_change',
                               'zip_code',
                               'witness_present',
                               'policy_report_filed_ind',
                               'vehicle_age',
                               'vehicle_color',
                               'fraud'], axis = 1)

#dropping these columns due to either lack of relevance and not appropriate format for powerBI

In [202]:
claim_bi_df.head()

,claim_number,gender,safety_rating,living_status,claim_date,claim_day_of_week,accident_site,no._of_claims,liab(%),channel,...,day_coded,month,witness_present_cat,marital_status_cat,college_graduate_cat,address_change_cat,safety_rating_cat,liab(%)_cat,policy_report,fraud_or_not
0,1,M,85,Rent,2016-12-16,Friday,Local,1,74,Broker,...,6,dec,not present,Married,college graduate,address changed,81 to 100,61 to 80,incomplete,legitimate
1,3,F,75,Rent,2015-02-12,Thursday,Highway,1,79,Online,...,5,feb,present,Not Married,not a college graduate,address changed,61 to 80,61 to 80,incomplete,legitimate
2,4,F,87,Own,2016-12-06,Tuesday,Local,0,0,Broker,...,3,dec,not present,Not Married,not a college graduate,address changed,81 to 100,20 or less,incomplete,fraudulent
3,5,F,58,Own,2016-05-05,Thursday,Local,3,99,Broker,...,5,may,not present,Married,college graduate,not changed,41 to 60,81 to 100,complete,fraudulent
4,6,M,95,Rent,2015-10-27,Tuesday,Highway,0,7,Broker,...,3,oct,present,Married,college graduate,not changed,81 to 100,20 or less,incomplete,legitimate


In [203]:
claim_bi_df.head()

,claim_number,gender,safety_rating,living_status,claim_date,claim_day_of_week,accident_site,no._of_claims,liab(%),channel,...,day_coded,month,witness_present_cat,marital_status_cat,college_graduate_cat,address_change_cat,safety_rating_cat,liab(%)_cat,policy_report,fraud_or_not
0,1,M,85,Rent,2016-12-16,Friday,Local,1,74,Broker,...,6,dec,not present,Married,college graduate,address changed,81 to 100,61 to 80,incomplete,legitimate
1,3,F,75,Rent,2015-02-12,Thursday,Highway,1,79,Online,...,5,feb,present,Not Married,not a college graduate,address changed,61 to 80,61 to 80,incomplete,legitimate
2,4,F,87,Own,2016-12-06,Tuesday,Local,0,0,Broker,...,3,dec,not present,Not Married,not a college graduate,address changed,81 to 100,20 or less,incomplete,fraudulent
3,5,F,58,Own,2016-05-05,Thursday,Local,3,99,Broker,...,5,may,not present,Married,college graduate,not changed,41 to 60,81 to 100,complete,fraudulent
4,6,M,95,Rent,2015-10-27,Tuesday,Highway,0,7,Broker,...,3,oct,present,Married,college graduate,not changed,81 to 100,20 or less,incomplete,legitimate


In [204]:
# claim_bi_df.to_csv('insurance_claims.csv')

In [205]:
# spark.stop()

# encoding variables - for ML 

In [206]:
claim_ml_df.columns

Index(['claim_number', 'gender', 'marital_status', 'safety_rating',
       'college_graduate', 'address_change', 'living_status', 'zip_code',
       'claim_date', 'claim_day_of_week', 'accident_site', 'no._of_claims',
       'witness_present', 'liab(%)', 'channel', 'policy_report_filed_ind',
       'claim_payout', 'vehicle_age', 'vehicle_category', 'vehicle_price',
       'vehicle_color', 'vehicle_weight', 'fraud', 'driver_age', 'salary',
       'age_range', 'salary_range', 'compensation_payout',
       'vehicle_price_range', 'year', 'month_coded', 'day_coded', 'month',
       'witness_present_cat', 'marital_status_cat', 'college_graduate_cat',
       'address_change_cat', 'safety_rating_cat', 'liab(%)_cat',
       'policy_report', 'fraud_or_not'],
      dtype='object')

## column removal - due to irrelevancy, not coded and/or inappropriate features such as date, year and month

In [207]:
claim_ml_df = claim_ml_df.drop(['claim_number',
                 'claim_date',
                 'claim_day_of_week',
                 'channel',
                 'vehicle_age',
                 'vehicle_category',
                 'vehicle_color',
                 'vehicle_weight',
                 'year',
                 'month_coded',
                 'day_coded',
                 'month',
                 'zip_code',
                 'fraud_or_not',
                 'witness_present_cat',
                 'marital_status_cat',
                 'college_graduate_cat',
                 'address_change_cat'], axis = 1)

In [208]:
claim_ml_df.head()

,gender,marital_status,safety_rating,college_graduate,address_change,living_status,accident_site,no._of_claims,witness_present,liab(%),...,fraud,driver_age,salary,age_range,salary_range,compensation_payout,vehicle_price_range,safety_rating_cat,liab(%)_cat,policy_report
0,M,1.0,85,1,1,Rent,Local,1,0.0,74,...,0,46,38301.0,44 to 52,31001 to 40000,4001 to 8000,5001 to 20000,81 to 100,61 to 80,incomplete
1,F,0.0,75,0,1,Rent,Highway,1,1.0,79,...,0,21,30445.0,17 to 25,21001 to 31000,1001 to 5000,15001 to 30000,61 to 80,61 to 80,incomplete
2,F,0.0,87,0,1,Own,Local,0,0.0,0,...,1,49,38923.0,44 to 52,31001 to 40000,4001 to 8000,15001 to 30000,81 to 100,20 or less,incomplete
3,F,1.0,58,1,0,Own,Local,3,0.0,99,...,1,58,40605.0,53 to 61,40001 to 50000,4001 to 8000,5001 to 20000,41 to 60,81 to 100,complete
4,M,1.0,95,1,0,Rent,Highway,0,1.0,7,...,0,38,36380.0,35 to 43,31001 to 40000,1001 to 5000,30001 to 45000,81 to 100,20 or less,incomplete


In [209]:
claim_ml_df.columns

Index(['gender', 'marital_status', 'safety_rating', 'college_graduate',
       'address_change', 'living_status', 'accident_site', 'no._of_claims',
       'witness_present', 'liab(%)', 'policy_report_filed_ind', 'claim_payout',
       'vehicle_price', 'fraud', 'driver_age', 'salary', 'age_range',
       'salary_range', 'compensation_payout', 'vehicle_price_range',
       'safety_rating_cat', 'liab(%)_cat', 'policy_report'],
      dtype='object')

## oneHot encoding: non-ordinal categories 

In [210]:
claim_ml_df_cats = claim_ml_df[['gender', 'living_status', 'accident_site', 'policy_report']]

claim_ml_df_cats = pd.DataFrame(claim_ml_df_cats)

In [211]:
dummy_cats = pd.get_dummies(claim_ml_df_cats)

## label encoding: ordinal categories


In [212]:


# claim_ml_df_label = claim_ml_df[['age_range', 'salary_range', 'compensation_payout', 'vehicle_price_range', 
#                                 'safety_rating_cat', 'liab(%)_cat']]

# claim_ml_df_label = pd.DataFrame(claim_ml_df_label)

# le = LabelEncoder()

# for col in claim_ml_df_label:
#     claim_ml_df_label[col] = le.fit_transform(claim_ml_df_label[col])
    
# claim_ml_df_label




In [213]:
claim_ml_df = claim_ml_df.drop(['age_range', 
                                'salary_range', 
                                'compensation_payout', 
                                'vehicle_price_range', 
                                'safety_rating_cat', 
                                'liab(%)_cat'], axis = 1)

In [214]:
fraud_target = claim_ml_df[['fraud']]

In [215]:
claim_ml_df.columns

Index(['gender', 'marital_status', 'safety_rating', 'college_graduate',
       'address_change', 'living_status', 'accident_site', 'no._of_claims',
       'witness_present', 'liab(%)', 'policy_report_filed_ind', 'claim_payout',
       'vehicle_price', 'fraud', 'driver_age', 'salary', 'policy_report'],
      dtype='object')

# further column dropping - Ml cannot have object based data 

In [216]:
# claim_ml_df = claim_ml_df.drop(['gender',
#                                'safety_rating',
#                                'living_status',
#                                'accident_site',
#                                'liab(%)',
#                                'policy_report_filed_ind',
#                                'claim_payout',
#                                'vehicle_price',
#                                'driver_age',
#                                'salary',
#                                'age_range',
#                                'salary_range',
#                                'compensation_payout',
#                                'vehicle_price_range',
#                                'safety_rating_cat',
#                                'liab(%)_cat',
#                                'policy_report'], axis = 1)

In [217]:
claim_ml_df

,gender,marital_status,safety_rating,college_graduate,address_change,living_status,accident_site,no._of_claims,witness_present,liab(%),policy_report_filed_ind,claim_payout,vehicle_price,fraud,driver_age,salary,policy_report
0,M,1.0,85,1,1,Rent,Local,1,0.0,74,0,7531.0,12885.0,0,46,38301.0,incomplete
1,F,0.0,75,0,1,Rent,Highway,1,1.0,79,0,2966.0,29429.0,0,21,30445.0,incomplete
2,F,0.0,87,0,1,Own,Local,0,0.0,0,0,6284.0,21701.0,1,49,38923.0,incomplete
3,F,1.0,58,1,0,Own,Local,3,0.0,99,1,6170.0,13198.0,1,58,40605.0,complete
4,M,1.0,95,1,0,Rent,Highway,0,1.0,7,0,4541.0,38060.0,0,38,36380.0,incomplete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,M,1.0,93,1,0,Own,Local,0,0.0,99,0,4807.0,32145.0,0,69,42338.0,incomplete
17994,F,0.0,22,1,1,Own,Local,1,0.0,94,1,4090.0,9469.0,1,35,35579.0,complete
17995,F,1.0,81,0,1,Rent,Local,2,0.0,1,1,2226.0,64974.0,0,27,32953.0,complete
17996,F,1.0,86,1,0,Own,Highway,1,1.0,75,1,9561.0,24983.0,0,52,39519.0,complete


## concatenating columns

In [218]:
claim_ml_df = pd.concat([claim_ml_df, dummy_cats], axis = 1)

In [219]:
claim_ml_df.columns

Index(['gender', 'marital_status', 'safety_rating', 'college_graduate',
       'address_change', 'living_status', 'accident_site', 'no._of_claims',
       'witness_present', 'liab(%)', 'policy_report_filed_ind', 'claim_payout',
       'vehicle_price', 'fraud', 'driver_age', 'salary', 'policy_report',
       'gender_F', 'gender_M', 'living_status_Own', 'living_status_Rent',
       'accident_site_Highway', 'accident_site_Local',
       'accident_site_Parking Lot', 'policy_report_complete',
       'policy_report_incomplete'],
      dtype='object')

In [220]:
claim_ml_df = claim_ml_df.drop(['gender',
                               'living_status',
                               'accident_site',
                               'fraud',
                               'policy_report',
                               'policy_report_filed_ind'], axis = 1)

In [221]:
claim_ml_df = pd.concat([claim_ml_df, fraud_target], axis = 1)

In [222]:
claim_ml_df.columns

Index(['marital_status', 'safety_rating', 'college_graduate', 'address_change',
       'no._of_claims', 'witness_present', 'liab(%)', 'claim_payout',
       'vehicle_price', 'driver_age', 'salary', 'gender_F', 'gender_M',
       'living_status_Own', 'living_status_Rent', 'accident_site_Highway',
       'accident_site_Local', 'accident_site_Parking Lot',
       'policy_report_complete', 'policy_report_incomplete', 'fraud'],
      dtype='object')

# Machine Learning Preparation

In [223]:
#shape and length of testing and training data 
def shape_and_length(train_data, test_data):
    print('shape:')
    print('\n')
    print('train data: ',train_data.shape)
    print('test data: ',test_data.shape)
    print('\n')
    print('Length:')
    print('\n')
    print('Train data length: ', len(train_data))
    print('Test data length: ', len(test_data))

In [224]:
#assessing column lengths 
len(claim_ml_df.columns)

21

In [225]:
#checking feature columns 
claim_ml_df.iloc[:,0:20]

,marital_status,safety_rating,college_graduate,address_change,no._of_claims,witness_present,liab(%),claim_payout,vehicle_price,driver_age,salary,gender_F,gender_M,living_status_Own,living_status_Rent,accident_site_Highway,accident_site_Local,accident_site_Parking Lot,policy_report_complete,policy_report_incomplete
0,1.0,85,1,1,1,0.0,74,7531.0,12885.0,46,38301.0,0,1,0,1,0,1,0,0,1
1,0.0,75,0,1,1,1.0,79,2966.0,29429.0,21,30445.0,1,0,0,1,1,0,0,0,1
2,0.0,87,0,1,0,0.0,0,6284.0,21701.0,49,38923.0,1,0,1,0,0,1,0,0,1
3,1.0,58,1,0,3,0.0,99,6170.0,13198.0,58,40605.0,1,0,1,0,0,1,0,1,0
4,1.0,95,1,0,0,1.0,7,4541.0,38060.0,38,36380.0,0,1,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,1.0,93,1,0,0,0.0,99,4807.0,32145.0,69,42338.0,0,1,1,0,0,1,0,0,1
17994,0.0,22,1,1,1,0.0,94,4090.0,9469.0,35,35579.0,1,0,1,0,0,1,0,1,0
17995,1.0,81,0,1,2,0.0,1,2226.0,64974.0,27,32953.0,1,0,0,1,0,1,0,1,0
17996,1.0,86,1,0,1,1.0,75,9561.0,24983.0,52,39519.0,1,0,1,0,1,0,0,1,0


In [226]:
# claim_ml_df.to_csv('ml_insurancefraud.csv')